In [1]:
!python -m pip install --quiet --upgrade pip
import torch
print(f"torch version: {torch.__version__}")

torch version: 1.7.1


In [3]:
!pip install --quiet -U transformers
import transformers
print(transformers.__version__)

4.11.3


In [7]:
from transformers import AutoConfig, AutoTokenizer, AutoModel

In [14]:
save_dir = "./model"
model_name_or_path = "sentence-transformers/paraphrase-xlm-r-multilingual-v1"

In [15]:
config = AutoConfig.from_pretrained(model_name_or_path)
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, config=config)

model = AutoModel.from_pretrained(
    model_name_or_path,
    from_tf=False,
    config=config,
)

In [16]:
config.save_pretrained(save_dir)
model.save_pretrained(save_dir)
tokenizer.save_pretrained(save_dir)

('./model/tokenizer_config.json',
 './model/special_tokens_map.json',
 './model/tokenizer.json')

### Package the pre-trained model and upload it to S3 ###

In [11]:
import os
import boto3
import sagemaker

role = sagemaker.get_execution_role()
sess = sagemaker.Session()

In [12]:
bucket = "ai-inference-env"
prefix = "sentence-similarity"

In [13]:
!ls -rtlh ./model/

total 1.1G
-rw-rw-r-- 1 ec2-user ec2-user  774 Oct 23 14:28 config.json
-rw-rw-r-- 1 ec2-user ec2-user  589 Oct 23 14:28 tokenizer_config.json
-rw-rw-r-- 1 ec2-user ec2-user  150 Oct 23 14:28 special_tokens_map.json
-rw-rw-r-- 1 ec2-user ec2-user 1.1G Oct 23 14:28 pytorch_model.bin
-rw-rw-r-- 1 ec2-user ec2-user 8.7M Oct 23 14:28 tokenizer.json


In [17]:
!cd model && tar czvf ../model.tar.gz *

config.json
pytorch_model.bin
special_tokens_map.json
tokenizer_config.json
tokenizer.json


In [18]:
fObj = open("model.tar.gz", "rb")
key = os.path.join(prefix, "model.tar.gz")
boto3.Session().resource("s3").Bucket(bucket).Object(key).upload_fileobj(fObj)
print(os.path.join(bucket, key))

ai-inference-env/sentence-similarity/model.tar.gz
